## Extreme Gradient Boosting

In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

import data_processing
import feature_extraction

In [2]:
df = data_processing.read_all_users('Geolife_data')

[1/38] processing user User37
[2/38] processing user User30
[3/38] processing user User31
[4/38] processing user User36
[5/38] processing user User38
[6/38] processing user User2
[7/38] processing user User5
[8/38] processing user User4
[9/38] processing user User3
[10/38] processing user User25
[11/38] processing user User22
[12/38] processing user User14
[13/38] processing user User13
[14/38] processing user User12
[15/38] processing user User15
[16/38] processing user User23
[17/38] processing user User24
[18/38] processing user User33
[19/38] processing user User34
[20/38] processing user User35
[21/38] processing user User32
[22/38] processing user User8
[23/38] processing user User6
[24/38] processing user User1
[25/38] processing user User7
[26/38] processing user User9
[27/38] processing user User21
[28/38] processing user User26
[29/38] processing user User19
[30/38] processing user User10
[31/38] processing user User17
[32/38] processing user User28
[33/38] processing user Us

In [16]:
df=data_processing.process_classes(df)

In [17]:
df = data_processing.create_segments(df)

In [18]:
df=data_processing.calculations(df)
df

,lat,lon,alt,elapsed time,label,time,trajectory,user,segment,distance,speed,time_diff,acceleration,bearing,heading_change,Vrate,angular_velocity,angular_acceleration
3,39.894600,116.332542,-777.0,39535.623056,train,2008-03-28 14:57:12,01020080328144824.plt,33,1,0.521027,0.008831,59.0,1.851318e-05,94.318545,1.058396e+01,1.236873e-01,1.793892e-01,2.464116e-03
4,39.889622,116.337040,-777.0,39535.623738,train,2008-03-28 14:58:11,01020080328144824.plt,33,1,0.673539,0.011416,59.0,4.381261e-05,145.266086,5.094754e+01,2.264335e-01,8.635176e-01,1.159540e-02
5,39.882090,116.338353,-777.0,39535.624421,train,2008-03-28 14:59:10,01020080328144824.plt,33,1,0.844980,0.014322,59.0,4.925049e-05,172.380803,2.711472e+01,2.028936e-01,4.595715e-01,-6.846546e-03
6,39.873873,116.338455,-777.0,39535.625116,train,2008-03-28 15:00:10,01020080328144824.plt,33,1,0.913730,0.015229,60.0,1.511915e-05,179.454178,7.073376e+00,5.956785e-02,1.178896e-01,-5.694698e-03
7,39.865182,116.338058,-777.0,39535.625799,train,2008-03-28 15:01:09,01020080328144824.plt,33,1,0.966989,0.016390,59.0,1.967467e-05,182.008052,2.553874e+00,7.082555e-02,4.328599e-02,-1.264468e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,40.029320,116.411975,289.0,39781.103808,train,2008-11-29 02:29:29,17920081129015805.plt,36,3851,0.023240,0.011620,2.0,-2.799201e-05,180.419823,2.068969e-01,4.817827e-03,-1.034485e-01,5.147249e-02
472,40.029111,116.411963,275.0,39781.103831,train,2008-11-29 02:29:31,17920081129015805.plt,36,3851,0.023262,0.011631,2.0,5.456289e-06,182.517368,2.097545e+00,9.382246e-04,1.048773e+00,5.761106e-01
473,40.028904,116.411962,274.0,39781.103854,train,2008-11-29 02:29:33,17920081129015805.plt,36,3851,0.023018,0.011509,2.0,-6.117035e-05,180.211944,2.305425e+00,1.063023e-02,-1.152712e+00,-1.100742e+00
474,40.028697,116.411961,274.0,39781.103877,train,2008-11-29 02:29:35,17920081129015805.plt,36,3851,0.023018,0.011509,2.0,-1.147129e-13,180.211944,6.431768e-07,1.993490e-11,3.215884e-07,5.763563e-01


In [19]:
feature_df = df.groupby('trajectory').apply(feature_extraction.extract_features).reset_index(drop=True)

/var/folders/6q/9kdss80974ldrx7hy_83wyn00000gn/T/ipykernel_38192/544797563.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  feature_df = df.groupby('trajectory').apply(feature_extraction.extract_features).reset_index(drop=True)


In [20]:
feature_df

,label,segment,trajectory,start_time,trajectory_length,total_distance,expected_speed,max_speed,mean_speed,top_1_speed,...,duration,average_acceleration,speed_std,acceleration_std,bearing_changes,HCR,stops,SR,VC,VCR
0,train,1,01020080328144824.plt,14,51,61.894997,0.020340,0.031892,60.059836,0.031892,...,1.030556,3.389920e-06,0.011135,0.000059,7,0.137255,3,0.058824,15,0.294118
1,car,2,01020080617094444.plt,12,367,3.459642,0.008484,0.016202,30.676631,0.016202,...,0.112778,-1.364201e-05,0.004158,0.001112,29,0.079019,0,0.000000,52,0.141689
2,train,3,01020080618121033.plt,15,1506,32.994611,0.019507,0.034116,66.506495,0.034116,...,0.496111,2.153045e-05,0.007654,0.000693,8,0.005312,0,0.000000,15,0.009960
3,train,4,01020080618160000.plt,0,9339,263.807927,0.023009,0.034759,70.099538,0.034759,...,3.763333,2.229937e-06,0.009790,0.000530,395,0.042296,343,0.036728,494,0.052896
4,train,5,01020080801023537.plt,15,2177,64.935795,0.026522,0.039892,95.376932,0.039892,...,0.680833,6.046446e-06,0.008630,0.000663,8,0.003675,0,0.000000,19,0.008728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3624,bike,3847,17920081115105653.plt,13,517,2.648993,0.002535,0.014397,9.222799,0.014397,...,0.287222,7.261407e-07,0.001392,0.000624,221,0.427466,2,0.003868,230,0.444874
3625,walk,3848,17920081116065153.plt,7,322,0.897082,0.001393,0.006847,5.030368,0.006847,...,0.178333,-6.023325e-07,0.000796,0.000385,130,0.403727,7,0.021739,162,0.503106
3626,walk,3849,17920081117025429.plt,2,194,0.558652,0.001431,0.005870,5.183372,0.005870,...,0.107778,6.367533e-07,0.000802,0.000421,74,0.381443,2,0.010309,90,0.463918
3627,walk,3850,17920081117030421.plt,7,240,0.723800,0.001495,0.004844,5.317717,0.004844,...,0.136111,9.491422e-07,0.000798,0.000435,101,0.420833,11,0.045833,135,0.562500


Encode labels

In [21]:
encoder = LabelEncoder()
feature_df['label'] = encoder.fit_transform(feature_df['label'])  # Encodes labels into numbers

Train/Test

In [22]:
# Define features and target, drop trajectory column
X = feature_df.drop(columns=['label', 'trajectory'])  # Replace 'label' with actual target column name
y = feature_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=36, stratify=y)
#with straitify: class distribution stays the same on train/test

Deal with class imbalance?

In [25]:
#??

Model

In [23]:
xgb_model = xgb.XGBClassifier(
    objective="multi:softmax", 
    num_class=len(set(y)),  # Set the number of classes for multi-class
    eval_metric="mlogloss",  # Use "logloss" for binary classification
    use_label_encoder=False,
    #scale_pos_weight=scale_pos_weight,  # Adjust weight for imbalance
    n_estimators=100,  # Number of trees
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)






Train the model

In [24]:
# Train model
xgb_model.fit(X_train, y_train)

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:36:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_class=5, num_parallel_tree=None, ...)

Predict the testset

In [25]:
y_pred = xgb_model.predict(X_test)

Evaluate

In [26]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.9173553719008265
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.91      0.91       158
           1       0.83      0.84      0.84        83
           2       0.91      0.90      0.91       124
           3       0.88      0.71      0.79        42
           4       0.94      0.97      0.96       319

    accuracy                           0.92       726
   macro avg       0.90      0.87      0.88       726
weighted avg       0.92      0.92      0.92       726

Confusion Matrix:
 [[143   3   2   0  10]
 [  6  70   2   1   4]
 [  0   7 112   2   3]
 [  1   2   7  30   2]
 [  5   2   0   1 311]]


# Tune Hyperparamters using gridsearch

Set parmaters to search

In [27]:
param_grid = {
    "n_estimators": [100, 200, 300],  # Number of boosting rounds
    "learning_rate": [0.01, 0.1, 0.2],  # Lower values prevent overfitting
    "max_depth": [4, 6, 8],  # Complexity of trees
    "min_child_weight": [1, 3, 5],  # Minimum sum of weights in a child node
    "subsample": [0.8, 1.0],  # Fraction of training samples used
    "colsample_bytree": [0.8, 1.0]  # Fraction of features used per tree
}

In [28]:
xgb_model = xgb.XGBClassifier(
    objective="multi:softmax", 
    num_class=len(set(y)),  # Set the number of classes for multi-class
    eval_metric="mlogloss",  # Use "logloss" for binary classification
    use_label_encoder=False,
    #scale_pos_weight=scale_pos_weight,  # Adjust weight for imbalance
    n_estimators=100,  # Number of trees
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

In [29]:
# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring="accuracy",
    cv=3,  # 3-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available CPUs
)

In [30]:
# Perform GridSearchCV
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:36:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:36:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:36:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:36:33] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packa

[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=1.0; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=1.0; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=1.0; total time=   0.7s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.8s


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:36:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:36:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:36:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:36:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packa

[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=200, subsample=0.8; total time=   1.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=4, min_child_weight=1, n_estimators=200, subsample=0.8; total time=   1.3s


...

[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=0.8; total time=   1.0s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=0.8; total time=   1.2s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   1.1s


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:40:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:40:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:40:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   1.2s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=200, subsample=1.0; total time=   1.1s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=0.8; total time=   1.2s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=0.8; total time=   1.1s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=0.8; total time=   1.0s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   1.0s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=8, min_child_weight=5, n_estimators=300, subsample=1.0; total time=   0.9s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_d

/opt/anaconda3/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:40:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=100,
                                     n_jobs=None, num_class=5,
                                     num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [4, 6, 8], 'min_child_weight': [1, 3, 5],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 1.0]},
             scoring='accuracy', verbose=2)

In [31]:
# Print the best parameters
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.8}
Best Cross-Validation Accuracy: 0.9111249754288204


In [32]:
best_params = grid_search.best_params_

best_xgb_model = xgb.XGBClassifier(
    objective="multi:softmax",  
    num_class=len(set(y_train)),  
    eval_metric="mlogloss",  
    use_label_encoder=False,
    random_state=42,
    **best_params  # Use best hyperparameters
)

best_xgb_model.fit(X_train, y_train)

# Predictions
y_pred = best_xgb_model.predict(X_test)


# Decode labels back to original transport mode names
y_test = encoder.inverse_transform(y_test)
y_pred = encoder.inverse_transform(y_pred)

# Evaluate model
print("Final Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:43:37] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Final Accuracy: 0.9173553719008265
Classification Report:
               precision    recall  f1-score   support

        bike       0.94      0.90      0.92       158
         bus       0.83      0.87      0.85        83
         car       0.90      0.90      0.90       124
       train       0.88      0.71      0.79        42
        walk       0.94      0.97      0.96       319

    accuracy                           0.92       726
   macro avg       0.90      0.87      0.88       726
weighted avg       0.92      0.92      0.92       726

Confusion Matrix:
 [[142   3   2   0  11]
 [  3  72   3   1   4]
 [  1   7 112   2   2]
 [  0   2   8  30   2]
 [  5   3   0   1 310]]


Use the large dataset

In [20]:
df = data_processing.read_all_users('geolife_trajectories')

[1/182] processing user 135
[2/182] processing user 132
[3/182] processing user 104
[4/182] processing user 103
[5/182] processing user 168
[6/182] processing user 157
[7/182] processing user 150
[8/182] processing user 159
[9/182] processing user 166
[10/182] processing user 161
[11/182] processing user 102
[12/182] processing user 105
[13/182] processing user 133
[14/182] processing user 134
[15/182] processing user 160
[16/182] processing user 158
[17/182] processing user 167
[18/182] processing user 151
[19/182] processing user 169
[20/182] processing user 156
[21/182] processing user 024
[22/182] processing user 023
[23/182] processing user 015
[24/182] processing user 012
[25/182] processing user 079
[26/182] processing user 046
[27/182] processing user 041
[28/182] processing user 048
[29/182] processing user 077
[30/182] processing user 083
[31/182] processing user 084
[32/182] processing user 070
[33/182] processing user 013
[34/182] processing user 014
[35/182] processing use

In [21]:
df = data_processing.process_classes(df)
df = data_processing.drop_unlabelled(df)

In [23]:
df = data_processing.calculations(df)
df = data_processing.create_segments(df)
feature_df = df.groupby('segment').apply(feature_extraction.extract_features).reset_index(drop=True)

/var/folders/6q/9kdss80974ldrx7hy_83wyn00000gn/T/ipykernel_35050/717751794.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  feature_df = df.groupby('segment').apply(feature_extraction.extract_features).reset_index(drop=True)


In [24]:
feature_df

,label,trajectory,start_time,trajectory_length,total_distance,average_speed,max_speed,duration,average_acceleration,speed_std,acceleration_std,bearing_changes,HCR,stops,SR,VC,VCR
0,bike,20070412101853.plt,10,2,0.013280,0.000511,0.000511,0.007222,NaN,NaN,NaN,0,0.000000,0,0.000000,0,0.000000
1,walk,20070412101853.plt,10,1,0.015877,0.000194,0.000194,0.000000,-0.000004,NaN,NaN,0,0.000000,1,1.000000,1,1.000000
2,bike,20070412102116.plt,10,54,2.744276,0.002698,0.009508,4.593333,0.000127,0.001779,0.000701,23,0.425926,7,0.129630,26,0.481481
3,walk,20070412102325.plt,10,3,1.587126,0.000212,0.000511,0.027222,0.000005,0.000303,0.000005,2,0.666667,2,0.666667,2,0.666667
4,bike,20070412134621.plt,13,27,2.711323,0.003505,0.004522,0.813611,0.000006,0.001307,0.000026,8,0.296296,3,0.111111,4,0.148148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17665,bus,20111231075505.plt,7,123,2.985901,0.002529,0.012581,0.155556,0.000015,0.003291,0.000378,73,0.593496,37,0.300813,70,0.569106
17666,walk,20111231075505.plt,8,107,1.016969,0.001587,0.010779,0.155556,-0.000015,0.001942,0.000375,68,0.635514,17,0.158879,66,0.616822
17667,walk,20111231144437.plt,14,80,13.136109,0.001587,0.028927,0.188889,0.000006,0.003174,0.000897,47,0.587500,7,0.087500,38,0.475000
17668,car,20111231144437.plt,14,252,14.029307,0.011136,0.021326,0.343056,0.000006,0.006651,0.000316,45,0.178571,24,0.095238,59,0.234127


In [25]:
encoder = LabelEncoder()
feature_df['label'] = encoder.fit_transform(feature_df['label'])  # Encodes labels into numbers

In [27]:
# Define features and target, drop trajectory column
X = feature_df.drop(columns=['label', 'trajectory'])  
y = feature_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=36, stratify=y)

In [110]:
X_train

,start_time,trajectory_length,total_distance,average_speed,max_speed,duration,average_acceleration,speed_std,acceleration_std,bearing_changes,HCR,stops,SR,VC,VCR
3619,11,229,12.389912,0.001507,0.008465,3.015278,4.726470e-07,0.000862,0.000182,118,0.515284,8,0.034934,117,0.510917
77,22,537,1440.476136,0.023637,0.698681,16.302222,6.228582e-05,0.032120,0.002090,161,0.299814,30,0.055866,192,0.357542
136,14,85,2.613036,0.002236,0.026007,0.469722,2.211730e-04,0.003454,0.001548,32,0.376471,1,0.011765,53,0.623529
4144,0,5053,8.955003,0.002624,0.035475,0.572500,1.492501e-03,0.003694,0.007058,3631,0.718583,4,0.000792,35,0.006927
941,2,1183,8.010222,0.003769,0.017448,0.385000,3.603007e-06,0.001544,0.001257,200,0.169062,21,0.017751,253,0.213863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3475,10,217,16.667409,0.004564,0.019563,0.330556,1.422446e-04,0.004118,0.001696,106,0.488479,38,0.175115,99,0.456221
1101,16,356,4.916479,0.001802,0.006870,0.199722,1.832780e-06,0.000979,0.000523,210,0.589888,4,0.011236,196,0.550562
4055,1,14736,65.907542,0.008506,0.061640,13.165833,1.542662e-03,0.007597,0.009777,8060,0.546960,60,0.004072,82,0.005565
788,11,177,3.993803,0.003325,0.011500,0.145556,7.447984e-06,0.003153,0.000647,63,0.355932,23,0.129944,79,0.446328


Train model on best parameters of the small model

In [28]:
best_xgb_model = xgb.XGBClassifier(
    objective="multi:softmax",  
    num_class=len(set(y_train)),  
    eval_metric="mlogloss",  
    use_label_encoder=False,
    random_state=42,
    **best_params  # Use best hyperparameters
)

best_xgb_model.fit(X_train, y_train)

# Predictions
y_pred = best_xgb_model.predict(X_test)


# Decode labels back to original transport mode names
y_test = encoder.inverse_transform(y_test)
y_pred = encoder.inverse_transform(y_pred)

# Evaluate model
print("Final Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:14:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Final Accuracy: 0.8831352574985851
Classification Report:
               precision    recall  f1-score   support

        bike       0.92      0.87      0.89       236
         bus       0.82      0.80      0.81       474
         car       0.94      0.94      0.94      1128
       train       0.83      0.94      0.88       812
        walk       0.89      0.81      0.85       884

    accuracy                           0.88      3534
   macro avg       0.88      0.87      0.87      3534
weighted avg       0.88      0.88      0.88      3534

Confusion Matrix:
 [[ 205    5    1    0   25]
 [   9  380   47    5   33]
 [   1   55 1055    9    8]
 [   2    9   17  765   19]
 [   7   12    7  142  716]]
